In [1]:
import pygmo as pg
import numpy as np
import pandas as pd
from uda import ES, EP

In [2]:
%load_ext line_profiler

In [3]:
funcs = [1, 2, 6, 7, 9, 14]
dimensions = [2, 10, 30]
POP_SIZE = 20
N_RUNS = 51

## ES

In [17]:
results = []

for nx in dimensions:
    for func in funcs:
        generations =  nx * 10000 // POP_SIZE - 1
        
        generations = 100000 // POP_SIZE - 1
        N_RUNS = 8
        
        prob = pg.problem(pg.cec2014(prob_id=func, dim=nx))
        algo = ES(memory=True)
        archi = pg.archipelago(N_RUNS, algo=algo, prob=prob, pop_size=POP_SIZE)
        archi.evolve(generations)
        archi.wait()
        
        champions = [isl.get_population().champion_f[0] for isl in archi]
        evals = [isl.get_population().problem.get_fevals() for isl in archi]
        for run, properties in enumerate(zip(champions, evals)):
            result = {
                         "function": func,
                         "dimensions": nx,
                         "run": run,
                         "champion_f": properties[0],
                         "evals": properties[1]
                     }
            
            results.append(result)

results = pd.DataFrame(results, columns=["function", "dimensions", "run", "champion_f", "evals"])
results["error"] = results["champion_f"] - (results["function"] * 100)
results["success"] = results["error"] < 1e-8

In [20]:
pd.DataFrame(results.groupby(["function", "dimensions"]).mean().loc[:, ["success", "error"]])

,,success,error
function,dimensions,,
1,2,0.000,3184.604630
2,2,0.000,492.690871
6,2,0.000,0.000353
7,2,0.000,0.002692
9,2,0.375,1.368068
14,2,0.000,0.115004


## CMAES

In [4]:
results = []

for nx in dimensions:
    for func in funcs:
        generations = nx * 10000 // POP_SIZE - 1
        
        prob = pg.problem(pg.cec2014(prob_id=func, dim=nx))
        algo = pg.algorithm(pg.xnes(gen=generations, ftol=1e-9, xtol=1e-9))
        archi = pg.archipelago(N_RUNS, algo=algo, prob=prob, pop_size=POP_SIZE)
        archi.evolve(1)
        archi.wait()
        
        champions = [isl.get_population().champion_f[0] for isl in archi]
        evals = [isl.get_population().problem.get_fevals() for isl in archi]
        for run, properties in enumerate(zip(champions, evals)):
            result = {
                         "function": func,
                         "dimensions": nx,
                         "run": run,
                         "champion_f": properties[0],
                         "evals": properties[1]
                     }
            
            results.append(result)

results = pd.DataFrame(results, columns=["function", "dimensions", "run", "champion_f", "evals"])
results["error"] = results["champion_f"] - (results["function"] * 100)
results["success"] = results["error"] < 1e-8

In [5]:
pd.DataFrame(results.groupby(["function", "dimensions"]).mean().loc[:, ["success", "error"]])

success         error
function dimensions                        
1        2           1.000000  2.416960e-12
         10          1.000000  1.431418e-10
         30          1.000000  5.540171e-10
2        2           1.000000  2.716224e-12
         10          1.000000  1.049937e-10
         30          1.000000  5.704928e-10
6        2           0.823529  2.430716e-02
         10          0.000000  8.225581e+00
         30          0.000000  3.944331e+01
7        2           0.117647  1.161067e-02
         10          0.568627  4.784322e-03
         30          0.980392  1.450210e-04
9        2           0.588235  3.451163e-01
         10          0.019608  4.643142e+00
         30          0.000000  2.579088e+01
14       2           0.000000  2.570081e-01
         10          0.000000  4.131354e-01
         30          0.000000  3.708838e-01

## XNES

In [6]:
results = []

for nx in dimensions:
    for func in funcs:
        generations = nx * 10000 // POP_SIZE - 1
        
        prob = pg.problem(pg.cec2014(prob_id=func, dim=nx))
        algo = pg.algorithm(pg.xnes(gen=generations, ftol=1e-9, xtol=1e-9))
        archi = pg.archipelago(N_RUNS, algo=algo, prob=prob, pop_size=POP_SIZE)
        archi.evolve(1)
        archi.wait()
        
        champions = [isl.get_population().champion_f[0] for isl in archi]
        evals = [isl.get_population().problem.get_fevals() for isl in archi]
        for run, properties in enumerate(zip(champions, evals)):
            result = {
                         "function": func,
                         "dimensions": nx,
                         "run": run,
                         "champion_f": properties[0],
                         "evals": properties[1]
                     }
            
            results.append(result)

results = pd.DataFrame(results, columns=["function", "dimensions", "run", "champion_f", "evals"])
results["error"] = results["champion_f"] - (results["function"] * 100)
results["success"] = results["error"] < 1e-8

In [7]:
pd.DataFrame(results.groupby(["function", "dimensions"]).mean().loc[:, ["success", "error"]])

success         error
function dimensions                        
1        2           1.000000  3.387199e-12
         10          1.000000  1.794931e-10
         30          1.000000  6.350226e-10
2        2           1.000000  3.722687e-12
         10          1.000000  1.768315e-10
         30          1.000000  5.491565e-10
6        2           0.705882  4.410767e-02
         10          0.039216  7.958686e+00
         30          0.000000  3.978735e+01
7        2           0.078431  1.197181e-02
         10          0.607843  3.865747e-03
         30          1.000000  6.825602e-10
9        2           0.666667  1.899564e-01
         10          0.000000  4.155417e+00
         30          0.000000  2.479592e+01
14       2           0.000000  3.098500e-01
         10          0.000000  4.156407e-01
         30          0.000000  3.845845e-01

## Debug

### Algo

In [8]:
prob = pg.problem(pg.cec2014(prob_id=1, dim=2))
algo = ES(memory=True)
pop = pg.population(prob=prob, size=POP_SIZE)

In [9]:
%lprun -f algo.evolve algo.evolve(pop)

In [11]:
for _ in range(10):
    #print(pop.champion_f[0])
    print(pop.get_f()[:, 0].mean())
    algo.evolve(pop)

6787.639164846338
6517.004330737569
3911.1722927743176
3817.269752746578
12643.944862216365
7281.807480712583
5739.149739969055
8012.224507580415
3988.823754646437
5662.45915878652


### Island

In [12]:
prob = pg.problem(pg.cec2014(prob_id=1, dim=2))
algo = ES(memory=True)
isl = pg.island(algo=algo, prob=prob, size=POP_SIZE)

In [13]:
print(isl.get_population().champion_f[0])
isl.evolve(10)
isl.wait()
print(isl.get_population().champion_f[0])

96308580.35220265
2811.8345166483887
